In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Authenticate and create the PyDrive client.  #用來驗證
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#這邊的link是檔案共用連結
link1 = 'https://drive.google.com/open?id=1LXVF-Lke2i08Hrd5pZvs-YkJcISswHIY'  
link2 = 'https://drive.google.com/open?id=1a8Go5Zb0XP8-w5Donim37Zpw_HbWduQX'
link3 = 'https://drive.google.com/open?id=10GkVlMBhF2q5dnRRNBkY_E0ZWPzTn-Sq'  
link4 = 'https://drive.google.com/open?id=1QHXlAX2F0_Xszv-wxntZgpukka2hw8vj'
# Verify that you have everything after '='
fluff, id1 = link1.split('=') 
fluff, id2 = link2.split('=')
fluff, id3 = link3.split('=') 
fluff, id4 = link4.split('=') 


#這邊要放檔案名稱
downloaded1 = drive.CreateFile({'id':id1}) 
downloaded2 = drive.CreateFile({'id':id2}) 
downloaded3 = drive.CreateFile({'id':id3}) 
downloaded4 = drive.CreateFile({'id':id4}) 
downloaded1.GetContentFile('X_train') 
downloaded2.GetContentFile('Y_train') 
downloaded3.GetContentFile('X_test') 
downloaded4.GetContentFile('sample_submission') 
X_train = np.genfromtxt('X_train', delimiter=',', skip_header=1)
Y_train = np.genfromtxt('Y_train', delimiter=',', skip_header=1)
X_test = np.genfromtxt('X_test', delimiter=',', skip_header=1)
submission = pd.read_csv('sample_submission')

In [0]:
#先把特定行數做normalization
#選擇的行數為：0,1,3,4,5,7,10,12,25,26,27,28 , 先學助教
#這邊的mean與std要先存起來, 讓之後testing data可以用（用同一個標準做正規化才有用！！！！！）
def normalization(X, column, train = True ,X_mean = None, X_std = None):
    if train == True:
        #先建立好mean與std
        X_mean = np.mean(X[:, column],0)
        X_std  = np.std(X[:, column],0)
        X[:, column] = np.divide(np.subtract(X[:, column],X_mean), X_std)
        return X, X_mean, X_std
    
    X[:, column] = np.divide(np.subtract(X[:, column],X_mean), X_std)
    
    return X
  
  
  
def normalization_all(X, train = True, X_mean = None, X_std = None):
  if train == True:
    X_mean = np.mean(X, axis = 0)
    X_std = np.std(X, axis = 0)
    X = np.divide(np.subtract(X, X_mean), X_std)
    return X, X_mean, X_std
  X = np.divide(np.subtract(X, X_mean), X_std)
  return X
    

用 sklearn的 GradientBoosting試試看：

In [0]:
#normalization
X_train, X_mean, X_std = normalization_all(X_train)

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(learning_rate=0.25,subsample=0.65,random_state=10,n_estimators=70,max_depth=3,min_samples_split=8)
model.fit(X_train, Y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.25, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=8,
                           min_weight_fraction_leaf=0.0, n_estimators=70,
                           n_iter_no_change=None, presort='auto',
                           random_state=10, subsample=0.65, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [0]:
#X_test = normalization_all(X_test, train = False, X_mean = X_mean, X_std = X_std)
ans = model.predict(X_test)

In [0]:
for i in range(len(ans)):
  if ans[i] >= 0.5:
    submission.iloc[i,1] = 1
    
from google.colab import files
submission.to_csv('submit.csv', index = False)

files.download('submit.csv') 

這邊用DNN試試看是否可以work

In [0]:
#按照助教給的col進行normalization
col = [0,1,3,4,5,7,10,12,25,26,27,28]
X_train, X_mean, X_std = normalization(X_train, col, train = True)
Y_train = Y_train.reshape(-1,1)

In [0]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.4, random_state = 5)

In [0]:
from keras import regularizers
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping


epochs = 80
batch_size = 64

model = Sequential()
model.add(Dense(64, input_shape = (X_train.shape[1], )))
model.add(BatchNormalization(axis = 1, momentum=0.99, epsilon=0.001))
model.add(Activation('sigmoid'))
model.add(Dropout(0.4))

model.add(Dense(64))
model.add(BatchNormalization(axis = 1, momentum=0.99, epsilon=0.001))
model.add(Activation('sigmoid'))
model.add(Dropout(0.4))

model.add(Dense(64))
model.add(BatchNormalization(axis = 1, momentum=0.99, epsilon=0.001))
model.add(Activation('sigmoid'))
model.add(Dropout(0.4))

model.add(Dense(64))
model.add(BatchNormalization(axis = 1, momentum=0.99, epsilon=0.001))
model.add(Activation('sigmoid'))
model.add(Dropout(0.4))

model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adamax', metrics=['binary_accuracy'])


In [0]:
#callbacks_list = [EarlyStopping(monitor = 'binary_accuracy', mode = max, patience = 2)]
train_histroy = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size = batch_size, epochs = epochs)

Train on 19536 samples, validate on 13025 samples
Epoch 1/80
19536/19536 [==============================] - 5s 243us/step - loss: 0.3639 - binary_accuracy: 0.8242 - val_loss: 0.3329 - val_binary_accuracy: 0.8495
Epoch 2/80
19536/19536 [==============================] - 5s 242us/step - loss: 0.3641 - binary_accuracy: 0.8237 - val_loss: 0.3373 - val_binary_accuracy: 0.8516
Epoch 3/80
19536/19536 [==============================] - 5s 243us/step - loss: 0.3633 - binary_accuracy: 0.8256 - val_loss: 0.3319 - val_binary_accuracy: 0.8526
Epoch 4/80
19536/19536 [==============================] - 5s 240us/step - loss: 0.3630 - binary_accuracy: 0.8306 - val_loss: 0.3328 - val_binary_accuracy: 0.8524
Epoch 5/80
19536/19536 [==============================] - 5s 239us/step - loss: 0.3596 - binary_accuracy: 0.8290 - val_loss: 0.3309 - val_binary_accuracy: 0.8515
Epoch 6/80
19536/19536 [==============================] - 5s 247us/step - loss: 0.3639 - binary_accuracy: 0.8264 - val_loss: 0.3338 - val_bi

In [0]:
X_test = normalization(X_test, col, train = False, X_mean = X_mean, X_std = X_std)

In [0]:
type(X_test)

numpy.ndarray

In [0]:
ans = model.predict(X_test)
for i in range(len(ans)):
  if ans[i] >= 0.5:
    submission.iloc[i,1] = 1
    
#注意, 很多答案都是同一個, 在ans裡面（感覺不是很合理）

In [0]:
from google.colab import files
submission.to_csv('submit.csv', index = False)

In [0]:
files.download('submit.csv') 